# Autonomous Claims Adjudication - Multi-Agent Sequential Pattern with Strands Agents on AgentCore

### Autonomous Claims Adjudication

Claims adjudication is the process by which insurance companies process, review, validate, and assess claims to determine if they should be paid, adjusted, or denied.

Here is a Generic Example of a Claim Adjudication Workflow for Auto-insurnace

NOTE: THIS IS JUST AN EXAMPLE FOR ILLUSTRATION PURPOSES. 




<p align="center">
    <img src="./images/ClaimsSequentialFlow.png">
</p>

### Agentic Pattern - Sequential

In the Claims Adjudication Flow, there are tasks that have dependencies and   need to be completed before moving to the next task.
We can use a Swarm Sequential pattern where each agent completes its task before passing the result to the next agent in the chain. 


#### Agentic Flow:
An agent that receives an insurance claim (First Notification of Loss), retrieves policy details, validates information against external sources (e.g., repair shop estimates), completes appraisal or damage assessments, and approves payment or flags the claim for human review.

Pattern —> Workflow with Sequential process


<p align="center">
    <img src="./images/ClaimsStrands.drawio.png">
</p>

In [ ]:
%pip install -q -r requirements.txt --no-cache-dir --force-reinstall

In [1]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [6]:
!pip freeze | grep boto
!pip freeze | grep agentcore

aioboto3 @ file:///home/conda/feedstock_root/build_artifacts/aioboto3_1742196379442/work
aiobotocore @ file:///home/conda/feedstock_root/build_artifacts/aiobotocore_1741606508148/work
boto3==1.42.49
botocore==1.42.49
opentelemetry-instrumentation-boto==0.60b1
bedrock-agentcore==1.3.0
bedrock-agentcore-starter-toolkit==0.2.10


In [20]:
# Import libraries
import os
import json
import requests
import boto3
from boto3.session import Session
from strands.tools import tool

# Get boto session
boto_session = Session()
region = boto_session.region_name

In [21]:
%store -r

### HELPER FUNCTIONS

In [22]:
import logging
from uuid import uuid4
from urllib.parse import quote

logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

def fetch_agent_card(agent_arn):
    # URL encode the agent ARN
    escaped_agent_arn = quote(agent_arn, safe='')

    # Construct the URL
    url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations/.well-known/agent-card.json"
    logger.info(url)
    # Generate a unique session ID
    session_id = str(uuid4())
    logger.info(f"Generated session ID: {session_id}")

    # Set headers
    headers = {
        'Accept': '*/*',
        'Authorization': f'Bearer {bearer_token}',
        'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id,
        'X-Amzn-Trace-Id': f'aws_docs_assistant_{session_id}'
    }

    try:
        # Make the request
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Parse and pretty print JSON
        agent_card = response.json()
        logger.info(json.dumps(agent_card, indent=2))

        return agent_card

    except requests.exceptions.RequestException as e:
        logger.error(f"Error fetching agent card: {e}")
        return None

In [23]:
import asyncio
import logging
import os
from uuid import uuid4

import httpx
from a2a.client import A2ACardResolver, ClientConfig, ClientFactory
from a2a.types import Message, Part, Role, TextPart

logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

DEFAULT_TIMEOUT = 300  # set request timeout to 5 minutes

def format_agent_response(response):
    """Extract and format agent response for human readability."""
    # Get the main response text from artifacts
    if response.artifacts and len(response.artifacts) > 0:
        artifact = response.artifacts[0]
        if artifact.parts and len(artifact.parts) > 0:
            return artifact.parts[0].root.text
    
    # Fallback: concatenate all agent messages from history
    agent_messages = [
        msg.parts[0].root.text 
        for msg in response.history 
        if msg.role.value == 'agent' and msg.parts
    ]
    return ''.join(agent_messages)


def create_message(*, role: Role = Role.user, text: str) -> Message:
    return Message(
        kind="message",
        role=role,
        parts=[Part(TextPart(kind="text", text=text))],
        message_id=uuid4().hex,
    )

async def send_sync_message(agent_arn, message: str):
    # Get runtime URL from environment variable
    escaped_agent_arn = quote(agent_arn, safe='')

    # Construct the URL
    runtime_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations/"
    
    # Generate a unique session ID
    session_id = str(uuid4())
    print(f"Generated session ID: {session_id}")

    # Add authentication headers for AgentCore
    headers = {"Authorization": f"Bearer {bearer_token}",
              'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id}
        
    async with httpx.AsyncClient(timeout=DEFAULT_TIMEOUT, headers=headers) as httpx_client:
        # Get agent card from the runtime URL
        resolver = A2ACardResolver(httpx_client=httpx_client, base_url=runtime_url)
        agent_card = await resolver.get_agent_card()
        print(agent_card)

        # Agent card contains the correct URL (same as runtime_url in this case)
        # No manual override needed - this is the path-based mounting pattern

        # Create client using factory
        config = ClientConfig(
            httpx_client=httpx_client,
            streaming=False,  # Use non-streaming mode for sync response
        )
        factory = ClientFactory(config)
        client = factory.create(agent_card)

        # Create and send message
        msg = create_message(text=message)

        # With streaming=False, this will yield exactly one result
        async for event in client.send_message(msg):
            if isinstance(event, Message):
                logger.info(event.model_dump_json(exclude_none=True, indent=2))
                return event
            elif isinstance(event, tuple) and len(event) == 2:
                # (Task, UpdateEvent) tuple
                task, update_event = event
                logger.info(f"Task: {task.model_dump_json(exclude_none=True, indent=2)}")
                if update_event:
                    logger.info(f"Update: {update_event.model_dump_json(exclude_none=True, indent=2)}")
                return task
            else:
                # Fallback for other response types
                logger.info(f"Response: {str(event)}")
                return event

In [24]:
# Load FNOL JSON data
with open('data/FNOL-DenyExample.json', 'r') as f:
    fnol_data = json.load(f)

print("✓ FNOL data loaded successfully")
print(f"\nClaim Number: {fnol_data['fnol']['claimNumber']}")
print(f"Report Date: {fnol_data['fnol']['reportDate']}")
print(f"Policyholder: {fnol_data['policyholder']['firstName']} {fnol_data['policyholder']['lastName']}")
print(f"Vehicle: {fnol_data['vehicle']['insuredVehicle']['year']} {fnol_data['vehicle']['insuredVehicle']['make']} {fnol_data['vehicle']['insuredVehicle']['model']}")

✓ FNOL data loaded successfully

Claim Number: CL-2023-1156789
Report Date: 2023-11-21
Policyholder: John Smith
Vehicle: 2020 Honda Accord


## 1. FNOL PROCESSING

In [25]:
# Create FNOL processing prompt
fnol_prompt = f"""
Please read the following FNOL (First Notice of Loss) data and extract all relevant information and check for inconsistencies.


FNOL Data:
{json.dumps(fnol_data, indent=2)}

You must only extract ll relevant infomration and do not provide any Claims approval or validation opinions.

"""

print("=" * 80)
print("PROCESSING FNOL DATA")
print("=" * 80)
print("\n FNOL prompt is ready ...\n")

PROCESSING FNOL DATA

 FNOL prompt is ready ...



In [26]:
from helpers.utils import reauthenticate_user

bearer_token = reauthenticate_user(
    FNOL_COGNITO_CLIENT_ID,
    FNOL_COGNITO_SECRET
)

In [27]:
# fetch_agent_card(FNOL_AGENT_ARN)

### Invoking FNOL Agent and Printing Output

In [28]:
result = await send_sync_message(FNOL_AGENT_ARN, fnol_prompt)
fnol_formatted_output = format_agent_response(result)
print(fnol_formatted_output)

Generated session ID: 2c9de0bc-57c0-4d61-bb04-d81be3a6ee6b
additional_interfaces=None capabilities=AgentCapabilities(extensions=None, push_notifications=None, state_transition_history=None, streaming=True) default_input_modes=['text'] default_output_modes=['text'] description='A Single agent with Claims Validation tools capabilities' documentation_url=None icon_url=None name='Claims Validator - ARC' preferred_transport='JSONRPC' protocol_version='0.3.0' provider=None security=None security_schemes=None signatures=None skills=[AgentSkill(description='Retrieves knowledge based on the provided text from Amazon Bedrock Knowledge Bases.\n\nKey Features:\n1. Semantic Search:\n   - Vector-based similarity matching\n   - Relevance scoring (0.0-1.0)\n   - Score-based filtering\n\n2. Advanced Configuration:\n   - Custom result limits\n   - Score thresholds\n   - Regional support\n   - Multiple knowledge bases\n\n3. Response Format:\n   - Sorted by relevance\n   - Includes metadata\n   - Source t

##  2. Policy Validation 

In [33]:
# Create user prompt with vehicle and damage information
validation_input_prompt = f"""
Validate this Claim request. Carefully read and use the {json.dumps(fnol_data, indent=2)} and  {fnol_formatted_output} provided to extract the required information to validate the claim


**fnol:** 
    reportDate
    reportTime": "14:35:00",
    reportMethod": "Online",
    submittedBy": "Policyholder"
    
**Claim Information:**
- Claim Number
- Incident Date
- Incident Description

**Insured Vehicle Information:**
- Year
- Make
- Model
- VIN
- Mileage: miles

**Damage Assessment:**
- Damaged Areas
- Damage Description
- Vehicle Drivable

Ouput: include a section that shows
## **CLAIM VALIDATION RESULT: INVALID vs VALID**
If RESULT is INVALID, provide explanation on why it was invalid and list the rules that were not met
"""

print("=" * 80)
print("USER PROMPT CREATED")
print("=" * 80)
# print(appraisal_input_prompt)
print("=" * 80)
print("\n📝 Prompt ready for agent execution")
print("\n" + "=" * 80)

USER PROMPT CREATED

📝 Prompt ready for agent execution



In [34]:
# CLAIMSVALIDATION_AGENT_ARN
AGENT1_AGENT_ARN# CLAIMSVALIDATION_COGNITO_CLIENT_ID 
# CLAIMSVALIDATION_COGNITO_SECRET 
# CLAIMSVALIDATION_DISCOVERY_URL

'arn:aws:bedrock-agentcore:us-east-1:161615149547:runtime/financial_analysis_assistant-mqZxfd4lJ4'

In [47]:
from helpers.utils import reauthenticate_user

bearer_token = reauthenticate_user(
    CLAIMSVALIDATION_COGNITO_CLIENT_ID ,
    CLAIMSVALIDATION_COGNITO_SECRET
)

In [48]:
validation_result = await send_sync_message(CLAIMSVALIDATION_AGENT_ARN, validation_input_prompt)
validation_formatted_output = format_agent_response(validation_result)
print(validation_formatted_output)

Generated session ID: b52d7963-e2cd-4895-9607-b19d0ba3cd92
additional_interfaces=None capabilities=AgentCapabilities(extensions=None, push_notifications=None, state_transition_history=None, streaming=True) default_input_modes=['text'] default_output_modes=['text'] description='A Single agent with Claims Validation tools capabilities' documentation_url=None icon_url=None name='Claims Validator - ARC' preferred_transport='JSONRPC' protocol_version='0.3.0' provider=None security=None security_schemes=None signatures=None skills=[AgentSkill(description='Retrieves knowledge based on the provided text from Amazon Bedrock Knowledge Bases.\n\nKey Features:\n1. Semantic Search:\n   - Vector-based similarity matching\n   - Relevance scoring (0.0-1.0)\n   - Score-based filtering\n\n2. Advanced Configuration:\n   - Custom result limits\n   - Score thresholds\n   - Regional support\n   - Multiple knowledge bases\n\n3. Response Format:\n   - Sorted by relevance\n   - Includes metadata\n   - Source t

##  3. APPRAISAL PROCESSING

In [38]:
# Create user prompt with vehicle and damage information
appraisal_input_prompt = f"""
Review {validation_formatted_output} and if ## **CLAIM VALIDATION RESULT: INVALID**, then stop the process and no additional APPRAISAL or SETTLEMENT is required.
If ## **CLAIM VALIDATION RESULT: VALID**, then estimate the repair costs for the following vehicle damage claim. Carefully read and use the {fnol_formatted_output} provided to extract the following required information:

**Claim Information:**
- Claim Number
- Incident Date
- Incident Description

**Insured Vehicle Information:**
- Year
- Make
- Model
- VIN
- Mileage: miles

**Damage Assessment:**
- Damaged Areas
- Damage Description
- Vehicle Drivable

Please provide a detailed repair cost estimate including:
1. Cost breakdown by damaged component
2. Any applicable cost adjustments (luxury brand, vehicle age, etc.)
3. Total estimated repair cost
4. Professional summary and recommendations
"""

print("=" * 80)
print("USER PROMPT CREATED")
print("=" * 80)
# print(appraisal_input_prompt)
print("=" * 80)
print("\n📝 Prompt ready for agent execution")
print("\n" + "=" * 80)

USER PROMPT CREATED

📝 Prompt ready for agent execution



In [39]:
from helpers.utils import reauthenticate_user

bearer_token = reauthenticate_user(
    APPRAISAL_COGNITO_CLIENT_ID,
    APPRAISAL_COGNITO_SECRET
)

In [40]:
# fetch_agent_card(APPRAISAL_AGENT_ARN)

### Invoking APPRAISAL agent and printing Output

Use the output of the FNOL Agent as context for input for the Appraisal Agent

In [41]:
appraisal_result = await send_sync_message(APPRAISAL_AGENT_ARN, appraisal_input_prompt)
appraisal_formatted_output = format_agent_response(appraisal_result)
print(appraisal_formatted_output)

Generated session ID: c036a0a2-d7a5-47b2-be1d-cf088f87e7ee
additional_interfaces=None capabilities=AgentCapabilities(extensions=None, push_notifications=None, state_transition_history=None, streaming=True) default_input_modes=['text'] default_output_modes=['text'] description='A Single agent wit Appraisal tools capabilities' documentation_url=None icon_url=None name='Repair Cost Estimator' preferred_transport='JSONRPC' protocol_version='0.3.0' provider=None security=None security_schemes=None signatures=None skills=[AgentSkill(description='Estimate repair costs for a collision-damaged vehicle based on vehicle type,\ndamaged components, and applicable cost multipliers.\n\nThis tool analyzes vehicle information and damage details to generate a comprehensive\nrepair cost estimate. It classifies the vehicle into a pricing tier (Car, Truck, SUV, Van),\ncalculates costs for each damaged component, applies multipliers for luxury brands and\nvehicle age, and returns a detailed JSON estimate.\n

## 3. SETTLEMENT ACTIONS
Use the output of the Apprasisal Agent as context for input for the Settlement Agent

In [42]:
bearer_token = reauthenticate_user(
    SETTLEMENT_COGNITO_CLIENT_ID,
    SETTLEMENT_COGNITO_SECRET
)

In [43]:
fetch_agent_card(SETTLEMENT_AGENT_ARN)

{'capabilities': {'streaming': True},
 'defaultInputModes': ['text'],
 'defaultOutputModes': ['text'],
 'description': 'Intelligent agent for automated auto insurance claims processing and settlement decisions',
 'name': 'AutoInsuranceClaimsSettlementAgent',
 'preferredTransport': 'JSONRPC',
 'protocolVersion': '0.3.0',
 'skills': [{'description': 'Verifies applicable coverage limits against policy terms.\n\nArgs:\n    claim_number: Unique claim identifier\n    estimated_cost: Total estimated repair cost\n    \nReturns:\n    Coverage verification details with policy limits and applicability',
   'id': 'verify_coverage_limits',
   'name': 'verify_coverage_limits',
   'tags': []},
  {'description': 'Applies policy deductible to calculate net insurance payout.\n\nArgs:\n    estimated_cost: Total repair estimate\n    deductible: Policy deductible amount\n    \nReturns:\n    Detailed breakdown of adjusted amounts',
   'id': 'apply_deductible_adjustments',
   'name': 'apply_deductible_adjust

In [44]:

settlement_prompt = f"""Please Settle the claim for the following claim appraisal: {appraisal_formatted_output}"""

### Invoking SETTLEMENT Agent and printing Ouput

In [45]:
settlement_result = await send_sync_message(SETTLEMENT_AGENT_ARN, settlement_prompt)
settlement_formatted_output = format_agent_response(settlement_result)
print(settlement_formatted_output)

Generated session ID: 814f10c5-aa48-4efc-9457-e96afb5ea672
additional_interfaces=None capabilities=AgentCapabilities(extensions=None, push_notifications=None, state_transition_history=None, streaming=True) default_input_modes=['text'] default_output_modes=['text'] description='Intelligent agent for automated auto insurance claims processing and settlement decisions' documentation_url=None icon_url=None name='AutoInsuranceClaimsSettlementAgent' preferred_transport='JSONRPC' protocol_version='0.3.0' provider=None security=None security_schemes=None signatures=None skills=[AgentSkill(description='Verifies applicable coverage limits against policy terms.\n\nArgs:\n    claim_number: Unique claim identifier\n    estimated_cost: Total estimated repair cost\n    \nReturns:\n    Coverage verification details with policy limits and applicability', examples=None, id='verify_coverage_limits', input_modes=None, name='verify_coverage_limits', output_modes=None, security=None, tags=[]), AgentSkill(de